## Solving set of polynomial equations using Homotopy Continuation

In [54]:
using HomotopyContinuation
using LinearAlgebra
using DynamicPolynomials

n = 2 # number of unknown variables

# Define parameters
δ = 0.1    # damping coefficient
ω = 1.5    # frequency
α = 1.0    # linear stiffness
β = 0.04   # nonlinearity
γ = 1.0    # driving force amplitude

# declare the variables 
for i in 1:n÷2
    @polyvar u[i] v[i]
end
@polyvar f[1:n]

# this input funcs will take Hew's function as an input containing all the polynomial equations
input_funcs = [(ω^2 - α)*u1 - δ*ω*v1 + γ - (3/4)*β*(u1^3 + u1*v1^2), ω*δ*u1 + (ω^2 - α)*v1 - (3/4)*β*(v1^3 + u1^2*v1)]

# define the variable f1 to have eqn 1 and f2 to have eqn 2
f = input_funcs

# Define the system as a polynomial system
system = HomotopyContinuation.System(f)

# solve the system
result = HomotopyContinuation.solve(system)

# # display non singular solutions
# for sol in result
#     if HomotopyContinuation.is_nonsingular(sol)
#         println(sol)
#     end
# end

# display real solutions
for sol in result
    if HomotopyContinuation.is_real(sol)
        println(sol)
    end
end



PathResult:
 • return_code → :success
 • solution → ComplexF64[1.3007933510765461 + 1.9983403963978888e-37im, 6.402380053229622 - 1.3518185034456306e-37im]
 • accuracy → 8.5864e-15
 • residual → 3.5527e-15
 • condition_jacobian → 164.32
 • steps → 47 / 0
 • extended_precision → false
 • path_number → 1

PathResult:
 • return_code → :success
 • solution → ComplexF64[-2.0000000000000093 + 3.3070643758065683e-43im, 5.9999999999999964 + 9.528829557408756e-44im]
 • accuracy → 2.6642e-15
 • residual → 8.8818e-16
 • condition_jacobian → 87.948
 • steps → 45 / 0
 • extended_precision → false
 • path_number → 5

PathResult:
 • return_code → :success
 • solution → ComplexF64[-0.800793351076541 - 1.4693679385278594e-39im, 0.09761994677037715 + 0.0im]
 • accuracy → 9.3417e-17
 • residual → 9.7578e-18
 • condition_jacobian → 1.2467
 • steps → 20 / 0
 • extended_precision → false
 • path_number → 7

